# FathomNet Python API Tutorial
*So you want to use FathomNet data...*

<img src="https://raw.githubusercontent.com/fathomnet/fathomnet-logo/main/FathomNet_white_CenterText_400px.png" alt="FathomNet logo" width="200"/>

## Introduction

> `fathomnet-py` is a client-side API to help scientists, researchers, and developers interact with FathomNet data.

[![tests](https://github.com/fathomnet/fathomnet-py/actions/workflows/tests.yml/badge.svg)](https://github.com/fathomnet/fathomnet-py/actions/workflows/tests.yml)
[![Documentation Status](https://readthedocs.org/projects/fathomnet-py/badge/?version=latest)](https://fathomnet-py.readthedocs.io/en/latest/?badge=latest)

The [fathomnet-py](https://github.com/fathomnet/fathomnet-py) API offers native Python interaction with the FathomNet REST API, abstracting away the underlying HTTP requests.
This notebook is designed to walk you through some of the core functionality of the API and to illustrate a common use case: *training an object detector*. 

It's split into two parts:
1. [**The API**](#api): API overview and data visualizations
2. [**Example use case**](#usecase): training an object detector and running inference

This notebook is by no means exhaustive; it serves to show some common "recipes" for pulling down and handling FathomNet data in Python. **Full documentation for fathomnet-py is available at [fathomnet-py.readthedocs.io](https://fathomnet-py.readthedocs.io).**

[FathomNet GitHub](https://github.com/fathomnet)

### Installing `fathomnet-py`

To install fathomnet-py, you will need to have Python 3.7 or greater installed first (as of the time of writing, this notebook ships with Python 3.9). Then, from the command-line:

```bash
pip install fathomnet
```

This notebook installs fathomnet-py in the [Setup](#setup) section next, along with some relevant packages for data manipulation and visualization.

<a name="setup"></a>
## Setup

First, we'll install a few packages via pip:

In [ ]:
!pip install -q -U fathomnet plotly ipyleaflet

and import the auxiliary modules we need for part 1:

In [ ]:
import ipywidgets as widgets                      # Provides embedded widgets
import ipyleaflet                                 # Provides map widgets
import requests                                   # Manages HTTP requests
import numpy as np                                # Facilitates array/matrix operations
import plotly.express as px                       # Generates nice plots
import random                                     # Generates pseudo-random numbers
from PIL import Image, ImageFont, ImageDraw       # Facilitates image operations
from io import BytesIO                            # Interfaces byte data

<a name="api"></a>
## The API

Now that we have fathomnet-py installed, let's see what it can do!

This section will show some of the common calls to pull down FathomNet data, and then we'll render some visualizations of the results.

### Overview

The two main parts of fathomnet-py are the **modules** and the **data models**.

#### Modules

fathomnet-py offers a variety of modules that encapsulate their relevant API operations. In brief:

- `boundingboxes` --- find & manage bounding boxes
- `darwincore` --- list owner institutions
- `images` --- find & manage images
- `geoimages` --- query for geo-images (geographic info only of images)
- `imagesetuploads` --- find & manage image set uploads
- `regions` --- list marine regions
- `stats` --- compute summary statistics
- `tags` --- find & manage custom image tags
- `taxa` --- get taxonomic information via a taxa provider
- `users` --- manage user accounts & list contributors
- `firebase` & `xapikey` -- authenticate for write-level operations

*Note: We will repeatedly import some of these modules in the notebook to highlight what's being used in each step. In your code, you only need to import a module once.*

Each operation (API call) is represented as a function in its given module. For example, to get an image by its universally-unique identifier (UUID), we can import the `fathomnet.api.images` module and call the `find_by_uuid` function.

In [ ]:
from fathomnet.api import images

example_image = images.find_by_uuid('79958ac5-832a-488c-9b48-cce7db346497')

#### Data models

To facilitate parsing and saving FathomNet data, native Python data models (dataclasses) are provided in the `fathomnet.models` module.

For example, we can see that the returned image from the `find_by_uuid` call above is of type `AImageDTO`.

In [ ]:
type(example_image)

These native data representations make it easier to write Python programs around FathomNet data. We'll print out some of the fields here.

In [ ]:
print('Image URL:', example_image.url)

print('Captured at latitude/longitude', example_image.latitude, example_image.longitude)

print('There are', len(example_image.boundingBoxes), 'bounding boxes:')
for box in example_image.boundingBoxes:
  print('-', box.concept, 'has area', box.width * box.height)

We can convert (serialize/deserialize) any of the FathomNet models to/from JSON or Python dictionaries. Let's print out the contents of that example image as JSON.

In [ ]:
print(example_image.to_json(indent=2))

### Bar chart of concepts with the most bounding boxes

Here we will use a `boundingboxes` operation, called `count_total_by_concept`, to get a quick count of the total number of bounding boxes for every concept in FathomNet. To visualize, we'll make a bar chart of the top `N`.

⚙ Try changing the value of `N` to show more concepts!

In [ ]:
from fathomnet.api import boundingboxes

# Make a bar chart of the top N concepts by bounding boxes
N = 5

# Get the number of bounding boxes for all concepts
concept_counts = boundingboxes.count_total_by_concept()

# Print out the total number of concepts
print('FathomNet has', len(concept_counts), 'localized concepts! Here are the top', N, 'by bounding box count.')

# Sort by number of bounding boxes
concept_counts.sort(key=lambda cc: cc.count, reverse=True)

# Get the top N concepts and their counts
concepts, counts = zip(*((cc.concept, cc.count) for cc in concept_counts[:N]))

# Make a bar chart
fig = px.bar(
    x=concepts, y=counts, 
    labels={'x': 'Concept', 'y': 'Bounding box count'}, 
    title=f'Top {N} concepts', 
    text_auto=True
)
fig.show()

### Listing images for a concept

Let's say we want to list all of the available images in FathomNet for a given concept. Here, we'll
1. List all the available concepts (again, using the `boundingboxes` module)
2. Pick one
3. Get a list of images for that concept using the `images` module

First, let's list all the available concepts in a choosable box.

We'll call the `find_concepts` function and put the results in a combo box.

⚙ **Pick a concept after running this cell!**

In [ ]:
from fathomnet.api import boundingboxes

# Get a list of all concepts that have at least 1 bounding box
all_concepts = boundingboxes.find_concepts()

# Pick one!
concept_combo = widgets.Combobox(
    options=all_concepts,
    description='Concept:',
    placeholder='Double-click or type here',
    ensure_option=True,
    disabled=False
)
concept_combo

With our concept selected (if you didn't put anything, it will default to *Chionoecetes tanneri*), we can call the `images` module `find_by_concept` function to get back a list of all images containing a bounding box for that concept.

In [ ]:
from fathomnet.api import images

# Get the selected concept
selected_concept = concept_combo.value
if not selected_concept:
  selected_concept = 'Chionoecetes tanneri'
  print('No concept selected. Using the default:', selected_concept)

# List the images FathomNet for that concept
concept_images = images.find_by_concept(selected_concept)

# Print the total number
print('Found', len(concept_images), 'images of', selected_concept)

This next cell will pick a random image, fetch it by its URL, and display it. 

⚙ If you want a different image, just re-run this cell.

In [ ]:
# Pick a random image
random_image = concept_images[random.randrange(len(concept_images))]

# Fetch and show the image
image_data = requests.get(random_image.url).content
pil_image = Image.open(BytesIO(image_data))
display(pil_image)

In [ ]:
# Concept -> color mapping for bounding boxes
def color_for_concept(concept: str):
  hash = sum(map(ord, concept)) << 5
  return f'hsl({hash % 360}, 100%, 85%)'

# Draw the bounding boxes and labels on the image
draw_image = ImageDraw.Draw(pil_image)
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf', size=18)
for box in random_image.boundingBoxes:
  color = color_for_concept(box.concept)
  draw_image.rectangle((box.x, box.y, box.x + box.width, box.y + box.height), width=3, outline=color)
  draw_image.text((box.x, box.y + box.height), box.concept, fill=color, font=font)

# Show the image with overlay
display(pil_image)

### Depth histogram

Let's generate a depth histogram; we'll extract the `depthMeters` field from each image (where present) and plot it.

In [ ]:
# Extract the depth (in meters) from each image
depths = [
    image.depthMeters
    for image in concept_images 
    if image.depthMeters is not None
]

# Make a horizontal histogram
fig = px.histogram(y=depths, title=f'{selected_concept} images by depth', labels={'y': 'depth (m)'})
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()

### Geographic heatmap

We can use the `latitude` and `longitude` fields to georeference each image. Here, we're generating a heatmap of the images overlaid on the Esri ocean basemap.

⚙ Zoom and pan around -- although the map is centered on the Monterey Bay, see if you can find where other "hotspots" are for your concept.

In [ ]:
# Extract the latitude/longitude from each image
locations = [
    (image.latitude, image.longitude)
    for image in concept_images
    if image.latitude is not None and image.longitude is not None
]

# Create a map from the Esri Ocean basemap
center = (36.807, -121.988)  # Monterey Bay
map = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.OceanBasemap, 
    center=center, 
    zoom=10
)
map.layout.height = "800px"

# Overlay the image locations as a heatmap
heatmap = ipyleaflet.Heatmap(
    locations=locations,
    radius=20,
    min_opacity=0.5
)
map.add_layer(heatmap)

map

<a name="usecase"></a>
## Example use case

**Object detection**

### Get training data

Let's take a look at how we can leverage the Python API to download some images and bounding boxes from FathomNet. We'll use these images to train a model later-on, so make sure you've completed this section before proceeding in the notebook.

#### Query for what we want

As we saw before, we can use the `fathomnet.api.images` module to search for images by concept. But what if we need to fine-tune our query?

Let's make a laundry list...
- 50 images of *Gersemia juliepackardae*
- 50 images of anything > 1000 m in depth

We can find this data by specifying a *constraint object* in fathomnet-py, then calling a generalized image querying function. To do this, we need to grab `GeoImageConstraints` from the `fathomnet.models` module:

In [ ]:
from fathomnet.models import GeoImageConstraints

Now, we can make a set of constraints for each bullet point.

In [ ]:
gersemia_constraints = GeoImageConstraints(concept='Gersemia juliepackardae', limit=50)
min_depth_constraints = GeoImageConstraints(minDepth=1000.0, limit=50)

To query for image data according to these constraints, we'll call the `fathomnet.api.images.find` function.

In [ ]:
gersemia_images = images.find(gersemia_constraints)
print(f'Gersemia juliepackardae images: {len(gersemia_images)}')

min_depth_images = images.find(min_depth_constraints)
print(f'>1000m images: {len(min_depth_images)}')

The bounding boxes are included in the image data. Let's take a look at how many boxes we have per concept.

In [ ]:
# Collect unique images (make sure we don't have overlap)
all_images = []
all_images.extend(gersemia_images)
all_images.extend(im for im in min_depth_images if im not in all_images)

# Count how many boxes we have per concept
concept_counts = {}
for image in all_images:
  for box in image.boundingBoxes:
    if box.concept not in concept_counts:
      concept_counts[box.concept] = 0
    concept_counts[box.concept] += 1

for concept, count in sorted(concept_counts.items()):
  print(f'{count:4} {concept}')
print('-'*20)
print(f'{sum(concept_counts.values()):4} total')

Sweet! Now we know what we're working with. But, in order to get this data ready for training, we still need to do two things:
1. **Download** the images themselves
2. **Format** the bounding boxes into something the model can understand

#### Download the images

No magic here, we just need to download the images (via HTTP) to somewhere the notebook can find them.

In [ ]:
from urllib.request import urlretrieve
from urllib.parse import urlparse
from pathlib import Path
from progressbar import progressbar

# Create a directory for the images
data_dir = Path('/content/gersemia_voc')
image_dir = data_dir / 'JPEGImages'
image_dir.mkdir(exist_ok=True, parents=True)

# Download each image, saving each new file path to a list
image_paths = []
for image in progressbar(all_images, redirect_stdout=True):
  parsed_url = urlparse(image.url)
  url_path = Path(parsed_url.path)
  image_path = image_dir / Path(image.uuid).with_suffix(url_path.suffix)
  urlretrieve(image.url, image_path)
  image_paths.append(image_path)

#### Format the bounding boxes

We need to get the bounding boxes in a format the model can understand. Image data (`AImageDTO`) objects offer a convenience function to generate Pascal VOC annotations from their internal data. We can leverage this to quickly generate XML annotations of the form:

```xml
<annotation>
  <folder>images</folder>
  <filename>{image filename}</filename>
  <path>/content/drive/MyDrive/fathomnet-workshop-tests/images/{image filename}</path>
  <source>
    <database>FathomNet</database>
  </source>
  <size>
    <width>{image width}</width>
    <height>{image height}</height>
    <depth>3</depth>
  </size>
  <segmented>0</segmented>
  <object>
    <name>{concept}[ ({altConcept})]</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <occluded>0</occluded>
    <bndbox>
      <xmin>{x}</xmin>
      <xmax>{x + width}</xmax>
      <ymin>{y}</ymin>
      <ymax>{y + height}</ymax>
    </bndbox>
  </object>
  ...
</annotation>
```

In [ ]:
xml_dir = data_dir / 'Annotations'
xml_dir.mkdir(exist_ok=True, parents=True)

for image, image_path in zip(all_images, image_paths):
  xml_path = xml_dir / image_path.with_suffix('.xml').name

  pascal_voc = image.to_pascal_voc(path=str(image_path), pretty_print=True)
  xml_path.write_text(pascal_voc)

### Inference with a pre-trained model

There are loads of software tools out there to train and run deep learning models. For this tutorial we will use [Detectron2](https://github.com/facebookresearch/detectron2), Facebook Research's machine learning library. This is just one of many options. 

To start with, we'll need grab a bunch of software directly from GitHub

In [ ]:
!pip install -q -U 'git+https://github.com/facebookresearch/detectron2.git'

Double check that the fathomnet tools are installed.

In [ ]:
!pip install -q -U fathomnet

Then import all the needed libraries into the workspace.

In [ ]:
import detectron2                               # core deep learning library
import torchvision                              # library of datasets, models, and image transforms
import pickle                                   # serialization library data
import json                                     # data storage standard
import matplotlib.pyplot as plt                 # plotting utilities
import torch                                    # tensor library for manipulating large models and data
import requests                                 # Manages HTTP requests
import random                                   # random number generator
import numpy as np                              # array manipulations
from skimage import io                          # use skimage to load individual images from a url
from fathomnet.api import images, boundingboxes # use fathomnet to find images

# import specific functions from detectron for shorter syntax when calling
from detectron2 import model_zoo
from detectron2.data import Metadata
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer

# import specific libraries from pyplot and PIL for easy plotting
from matplotlib.pyplot import imshow
from PIL import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#### Download a model from the FathomNet model zoo
A big feature of FathomNet is the *ModelZoo*, a repository for users to share their models with the community. For the moment, [we are advising users](https://medium.com/fathomnet/how-to-upload-your-ml-model-to-fathomnet-68b933dd55bd) to upload their models on Zenodo to generate a DOI and then share them on our GitHub page.  We have provided a number of our models as a starting point. 

For this section of the workshop, we'll download the [MBARI Benthic Supercategory Detector](https://zenodo.org/record/5571043). This Retinanet model was fine tuned with FathomNet data from a version originally trained on COCO images. To train this system, we grouped many of our fine grained classes together into 20 'supercategories' that hopefully encode some generally morphological informatoin about the group. All the training data was drawn from MBARI imagery collected in Monterey Bay. 

We will need to make sure that Colab can talk to your personal Google Drive in order to download training images, pretrained model weights, etc. This command will open a pop-up window asking you which account link with and for permission to access files. 

In [ ]:
from google.colab import drive

try:
  drive.mount('/content/drive')
except ValueError:
  pass

We will run `wget` to actually do the download. This command will let Colab download resources from a URL. We will start by getting the weights from the 

The syntax for `wget` is: 

```
wget <url-to-download> --directory-prefix=<content/drive/MyDrive/my-workshop-directory>
```

The first set of brackets points to the desired URL, the second is the location of the output directory on your drive. 

First, let's download the model weights. 

In [ ]:
!wget -nc https://zenodo.org/record/5571043/files/model_final.pth --directory-prefix=/content/drive/MyDrive/fathomnet-workshop-tests/

You should now be able to go into your Google Drive account and see the weights downloaded into your workshop directory.

Now we'll grab the model file that declares the structure of the model.

In [ ]:
!wget -nc https://zenodo.org/record/5571043/files/fathomnet_config_v2_1280.yaml --directory-prefix=/content/drive/MyDrive/fathomnet-workshop-tests/

#### Run inference
We can actually run images through our network now that we have the model architecture and the weights from training. Before we run anything we will need to load the model into memory and set several parameters that will dictate what we see in the output. 

First set the paths so the `detectron2` toolbox will know where to look for your files.

In [ ]:
CONFIG_FILE = "/content/drive/MyDrive/fathomnet-workshop-tests/fathomnet_config_v2_1280.yaml"   # training configuration file
WEIGHT_FILE = "/content/drive/MyDrive/fathomnet-workshop-tests/model_final.pth"                 # fathomnet model weights

Now set Non-Maximal Suppresion (NMS) and Score thresholds. These parameters dictate which of the proposed regions the algorithm displays. 

In [ ]:
NMS_THRESH = 0.45   # Set an NMS threshold to filter all the boxes proposed by the model
SCORE_THRESH = 0.3  # Set the model score threshold to suppress low confidence annotations

You have to explicitly tell the model what the names of the classes are. The system outputs a number, not a label. You can think of this as a look-up table.

In [ ]:
fathomnet_metadata = Metadata(
    name='fathomnet_val',
    thing_classes=[
         'Anemone',
         'Fish',
         'Eel',
         'Gastropod',
         'Sea star',
         'Feather star',
         'Sea cucumber',
         'Urchin',
         'Glass sponge',
         'Sea fan',
         'Soft coral',
         'Sea pen',
         'Stony coral',
         'Ray',
         'Crab',
         'Shrimp',
         'Squat lobster',
         'Flatfish',
         'Sea spider',
         'Worm'
    ]
)

With all the parameters and file paths set up, you can now point Detectron to the configurations using the `get_cfg()` function.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.merge_from_file(CONFIG_FILE)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = SCORE_THRESH
cfg.MODEL.WEIGHTS = WEIGHT_FILE 

Load in all the model weights and set the thresholds. This actually instantiates the model in your workspace. The `model` object is what will ingest the images and return outputs for us to look at. 

⚠ *If this cell returns a* `RuntimeError: No CUDA GPUs are available` *you will need to update your settings. Click the Runtime dropdown menu, select "Change runtime type" and select GPU in the "Hardware accelarator" box. You will then need to rerun the detectron2 install via pip.*  

In [ ]:
model = build_model(cfg)                      # returns a torch.nn.Module
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)          # This sets the weights to the pre-trained values dowloaded from Zenodo
model.eval()                                  # Tell detectron that this model will only run inference

Before putting images through the network, you need to define some preprocessing steps. At training time, you might set up a series of random affine transformations to help guard against overfitting. Since this network is already trained, we just need to resize time images to a standard dimension.

In [ ]:
aug = T.ResizeShortestEdge(
    short_edge_length=[cfg.INPUT.MIN_SIZE_TEST], 
    max_size=cfg.INPUT.MAX_SIZE_TEST, 
    sample_style="choice"
)

Finally, we need to set up an extra NMS layer since by default `detectron2` models only do intra-class comparisions between bounding boxes. We need to do another NMS run between classes. 

In [ ]:
post_process_nms = torchvision.ops.nms

We'll need to grab a random (or not so random) image to run through the network.

In [ ]:
# get a list of all concepts
all_concepts = boundingboxes.find_concepts()

# pick one at random (or set one yourself)
#concept = 'Chionoecetes tanneri'
concept = all_concepts[random.randrange(len(all_concepts))]

# Pull the images from FathomNet
concept_images = images.find_by_concept(concept)

print(f'{len(concept_images)} images of {concept}')

Finally, you have everything loaded up to run the image through the model.

In [ ]:
# Pick a random image
image = concept_images[random.randrange(len(concept_images))]

# Fetch the image
im = io.imread(image.url)

im_height,im_width,_ = im.shape  # grab the image dimensions

# Use detectron's visualization tool to plot the bounding boxes
v_inf = Visualizer(
    im,
    metadata=fathomnet_metadata, 
    scale=1.0, 
    instance_mode=ColorMode.IMAGE
)

# Transform the image in the desired input shape
im_transformed = aug.get_transform(im).apply_image(im)

# Actually crank it through the model
with torch.no_grad():
    im_tensor = torch.as_tensor(im_transformed.astype("float32").transpose(2, 0, 1))
    model_outputs = model([{"image" : im_tensor, "height" : im_height, "width" : im_width}])[0]

# Run the second stage NMS to ensure limited interclass overlap
model_outputs["instances"] = model_outputs["instances"][post_process_nms(model_outputs["instances"].pred_boxes.tensor, model_outputs["instances"].scores, NMS_THRESH).to("cpu").tolist()]

# Use the visualization tool to plot the bounding boxes on top of the image
out_inf_raw = v_inf.draw_instance_predictions(model_outputs["instances"].to("cpu"))

# Then display the output
fig, ax = plt.subplots(figsize=(24,16))
ax.imshow(out_inf_raw.get_image())
ax.set_axis_off()

### Train a model

Training a model takes time, computational resources, and interative evaluation. To illustrate the beginnings of this process, we will fine tune a (relatively) small object detector to find the soft coral *Gersemia juliepackardae* in benthic images.

We have already loaded the images in with a VOC file format. Detectron2 has a pre-made dataloader to finesse the dataset into the model training cycle. But first we need to specify lists of training and validation data. 

In [ ]:
import glob, os

all_imgs = glob.glob(os.path.join(data_dir / 'JPEGImages'))

print(data_dir / 'images')

In [ ]:
from detectron2.data.datasets import register_pascal_voc



register_pascal_voc("gersemia_train", data_dir, "train", class_names='Gersemia juliepackardae')